# MODELOS

#### Importar datos 

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#leer archivo limpio 
df = pd.read_csv('airbnb_rio_clean.csv')

### EXPLORACION

In [0]:

#explorar df
df.head()


In [0]:
df['room_type'].unique()

In [0]:
df.describe()

In [0]:
print(df.shape)
print( df.dtypes)
print(df.keys())

#### Definir variable dependiente del modelo

In [0]:
#Modelo de regresion--> variable y = precio . LISTA de nombre
y_reg= "price"

x_completo_reg=['host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication',"review_scores_rating", 'review_scores_location',
       'instant_bookable', 'bathroomsf', 'has_pool', 'has_wifi',
       'has_free_parking', 'has_ac_heating', 'has_kitchen', 'has_jacuzzi',
       'has_washer_dryer', 'has_self_checkin', 'has_tv_cable', 'has_bbq']
#Modelo de clasificacion --> variable y = rating . LISTA de nombre
y_clas= "review_scores_rating"

x_completo_class=['host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bedrooms', 'beds', 'price'
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication', 'review_scores_location',
       'instant_bookable', 'bathroomsf', 'has_pool', 'has_wifi',
       'has_free_parking', 'has_ac_heating', 'has_kitchen', 'has_jacuzzi',
       'has_washer_dryer', 'has_self_checkin', 'has_tv_cable', 'has_bbq']

#x categoricas vs continuas
continuas_reg=['host_response_rate', 'host_acceptance_rate',
       'host_total_listings_count', 'latitude',
       'longitude', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication',"review_scores_rating", 'review_scores_location',
        'bathroomsf']

continuas_class=['host_response_rate', 'host_acceptance_rate',
       'host_total_listings_count', 'latitude',
       'longitude', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication',"price", 'review_scores_location',
        'bathroomsf']

categoricas= ['host_is_superhost', 'host_identity_verified', 'room_type', 'instant_bookable', 
              'has_pool','has_wifi','has_free_parking',
              'has_ac_heating','has_kitchen','has_jacuzzi',
              'has_washer_dryer','has_self_checkin','has_tv_cable',
              'has_bbq']

#### Exploracion de variables X importante

In [0]:
#paiplot continuas
sns.pairplot(df, x_vars= continuas_reg, y_vars= y_reg, kind='reg') 

In [0]:
#pairplot discretas
sns.pairplot(df, x_vars= categoricas, y_vars= y_reg, kind='reg') 

#### Analisis variables continuas

In [0]:
df_continuo=df[['host_response_rate', 'host_acceptance_rate',
       'host_total_listings_count', 'latitude',
       'longitude', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication',"review_scores_rating", 'review_scores_location',
        'bathroomsf', 'price']]
#Ver correlacion entre datos 
corr = df_continuo.corr()[y_reg].sort_values(ascending=False)
#sns.heatmap(corr.to_frame(), cmap="Blues", annot=True)
print(corr)
# Ver dispersion y tendencia entre variables x y y
#Definir variables x totales. LISTA
sns.heatmap(df_continuo.corr())


##### Caclular VIF eliminar aquellas con VIF >10 

In [0]:
df_x=df.copy().drop(columns=['price'])

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame({
    "variable": df_x.columns,
    "VIF": [variance_inflation_factor(df_x.values, i) for i in range(df_x.shape[1])]
}).sort_values("VIF", ascending=False)

vif

In [0]:
# variables con VIF
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(df_x, thresh=10):
    variables = df_x.columns
    dropped = True

    while dropped:
        dropped = False
        vif = pd.DataFrame()
        vif["variable"] = variables
        vif["VIF"] = [variance_inflation_factor(df_x[variables].values, i)
                      for i in range(len(variables))]

        max_vif = vif['VIF'].max()
        if max_vif > thresh:
            max_var = vif.loc[vif['VIF'].idxmax(), 'variable']
            print(f"Eliminando {max_var} por VIF={max_vif}")
            variables = variables.drop(max_var)
            dropped = True

    print("Variables finales:")
    return variables

vars_finales = calculate_vif(df_x)

print(vars_finales)

In [0]:
#Variables con VIF<10
finales=['host_response_rate', 'host_is_superhost', 'host_total_listings_count',
       'host_identity_verified', 'room_type', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d', 'review_scores_rating', 'instant_bookable',
       'bathroomsf', 'has_pool', 'has_free_parking', 'has_ac_heating',
       'has_jacuzzi', 'has_washer_dryer', 'has_self_checkin', 'has_tv_cable',
       'has_bbq','price']
finales_continuas = [
    'host_response_rate',
    'host_total_listings_count',
    'bedrooms',
    'beds',
    'minimum_nights',
    'maximum_nights',
    'maximum_minimum_nights',
    'maximum_nights_avg_ntm',
    'availability_365',
    'availability_eoy',
    'estimated_occupancy_l365d',
    'review_scores_rating',
    'bathroomsf',
    'price'
]

finales_categoricas=['host_is_superhost',
 'host_identity_verified',
 'room_type',
 'instant_bookable',
 'has_pool',
 'has_free_parking',
 'has_ac_heating',
 'has_jacuzzi',
 'has_washer_dryer',
 'has_self_checkin',
 'has_tv_cable',
 'has_bbq']

df_x_final=df[finales_continuas]

In [0]:
corr = df_x_final.corr()[y_reg].sort_values(ascending=False)
print(corr)

## MODELO DE REGRESION

#### Seleccionar variables x iniciales

In [0]:

#crear nuevo dataframe con variables relevantes
datos=df[finales].copy()

#Dividir variables x numericas
x_num= [col for col in finales_continuas if col != 'price']
#Dividir variables x categoricas
x_cat=finales_categoricas

In [0]:
#Escoger variables que incluir 

#modelo 2
x_2= ["bathroomsf"]
x_num= ["bathroomsf"]

#modelo 4
x_4= ["bathroomsf", "bedrooms", "accommodates",'has_pool',] # No mejora tanto
x_num= ["bathroomsf","bedrooms","accommodates"]

#modelo 5
x_5= ["bathroomsf", "accommodates",'has_pool','has_self_checkin'] # MEjor que el 4
x_num= ["bathroomsf","accommodates"]

#modelo 6 #NO MEJORA
x_6= ["bathroomsf", "accommodates"]
x_num= ["bathroomsf","accommodates"]

#modelo 3 # base de 10.1
x_3= ["bathroomsf", "bedrooms", "accommodates"]
x_num= ["bathroomsf","bedrooms","accommodates"]

#modelo 7
x_7= ["bathroomsf", "accommodates",'has_pool','has_self_checkin', "room_type"]
x_num= ["bathroomsf","accommodates"]

#modelo 8
x_8= ["bathroomsf", "accommodates", "room_type"] #MEJOR QUE x_3
x_num= ["bathroomsf","accommodates"]

#modelo 9
x_9= ["bathroomsf","bedrooms", "accommodates", "room_type"]
x_num= ["bathroomsf","bedrooms","accommodates"]

#modelo 10
x_10= ["bathroomsf","bedrooms","room_type"]
x_num= ["bathroomsf","bedrooms"]





In [0]:
'0. Funcion para estandarizar los datos numericos solo'
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

preprocesamiento = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), x_num),   # escalo solo las continuas
        # las categóricas (0/1) pasan sin tocarse
    ],remainder="passthrough")

'1. Dividir test y train'
#  Definir X e y
y = df["price"].copy()
X = df[x_10].copy() # CAMBIARRRRRRR

# 4. Dividir en train y test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True )

'2. Estandarizar x num (se asume que categoricos ya estan en dummies)'
# Ajustar el preprocesador con datos de entrenamiento
preprocesamiento.fit(X_train)

# Transformar train y test --> estandariza
X_train_proc = preprocesamiento.transform(X_train)
X_test_proc = preprocesamiento.transform(X_test)

In [0]:
print("Media por columna:", np.mean(X_train_proc, axis=0))
print("Std por columna:", np.std(X_train_proc, axis=0))

### CREAR MLFLOW

In [0]:
# Imports
import mlflow
import mlflow.sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error, mean_squared_error,)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# MLflow
mlflow.set_registry_uri("databricks-uc")
experiment = mlflow.set_experiment("/regresionisabella")

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="modelo 10.10.log"):

    # ----- Hiperparámetros -----
    params_nn = {
        "hidden_layer_sizes": (64, 32, 16, 8),
        "activation": "relu", #check
        "solver": "adam",
        "learning_rate_init": 0.005,
        "max_iter": 500,
        "random_state": 42,
    }
    mlflow.log_params(params_nn)

    # ----- tranformar y -> log(y) -----
    y_train_log = np.log(y_train)

    # ----- Modelo ----------------------------
    nn = MLPRegressor(**params_nn)

    #  ----- entrenar usando x y y transformados -----
    nn.fit(X_train_proc, y_train_log)

    #  ----- hacer predicciones y revertir escala de las predicciones log(y)-> y -----
    pred_log = nn.predict(X_test_proc)
    predictions = np.exp(pred_log)

    # ----- Guardar número de iteraciones ---------------------
    with open("n_iter.txt", "w") as f:
        f.write(str(nn.n_iter_))

    mlflow.log_artifact("n_iter.txt")


    # ----- Guardar loss curve -----
    plt.figure()
    plt.plot(nn.loss_curve_)
    plt.xlabel("Iteraciones")
    plt.ylabel("Loss")
    plt.title("Curva de pérdida (loss_curve_)")
    plt.savefig("loss_curve.png", bbox_inches="tight")
    plt.close()

    mlflow.log_artifact("loss_curve.png")

    # ----- DataFrame auxiliar -----
    df_plot = pd.DataFrame({"y_real": y_test, "pred": predictions})
    df_plot["error_abs"] = (df_plot["pred"] - df_plot["y_real"]).abs()

    # 1) Error vs precio
    plt.figure()
    plt.scatter(df_plot["y_real"], df_plot["error_abs"], alpha=0.3)
    plt.xlabel("Precio real")
    plt.ylabel("Error absoluto")
    plt.title("Error absoluto vs precio real")
    plt.savefig("error_vs_price.png", bbox_inches="tight")
    plt.close()
    mlflow.log_artifact("error_vs_price.png")

    # 2) MAE por cuartil
    df_plot["price_bin"] = pd.qcut(df_plot["y_real"], 4, labels=["Q1", "Q2", "Q3", "Q4"])
    quartile_mae = df_plot.groupby("price_bin")["error_abs"].mean()
    quartile_mae.to_csv("mae_por_cuartil.csv")
    mlflow.log_artifact("mae_por_cuartil.csv")

    # 3) Real vs predicho
    plt.figure()
    plt.scatter(df_plot["y_real"], df_plot["pred"], alpha=0.3)
    min_val = min(df_plot["y_real"].min(), df_plot["pred"].min())
    max_val = max(df_plot["y_real"].max(), df_plot["pred"].max())
    plt.plot([min_val, max_val], [min_val, max_val], "r-")
    plt.xlabel("Real")
    plt.ylabel("Predicho")
    plt.title("Real vs Predicho")
    plt.savefig("real_vs_pred.png", bbox_inches="tight")
    plt.close()
    mlflow.log_artifact("real_vs_pred.png")

    # Features como artifact
    with open("features.txt", "w") as f:
        for col in X_train.columns:
            f.write(f"{col}\n")
    mlflow.log_artifact("features.txt")

    # ----- Métricas -----
    mse = mean_squared_error(y_test, predictions)
    rmse = mse**0.5
    mae = mean_absolute_error(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    metrics = {"mse": mse, "rmse": rmse, "mae": mae, "mape": mape}
    for name, value in metrics.items():
        mlflow.log_metric(name, value)
        print(f"{name.upper()}: {value}")

    # ----- Registrar modelo -----
    mlflow.sklearn.log_model(nn, "nn-regression-model")



### Modelos de clasificacion- alquilar o no